In [1]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [2]:
merged = pd.read_csv(path+'data/merged.csv')

In [26]:
test = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality', 'driver_wins_after_race', 'driver_standings_pos_after_race',  'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1)

In [27]:
driver_points_per_race = []
driver_points_from_race = []

constructor_points_per_race = []
constructor_points_from_race = []

for index, row in test.iterrows():
    driver_points_per_race.append(round(row.driver_points / row['round'], 2))
    driver_points_from_race.append(round(row.driver_points_after_race - row.driver_points, 1))

    constructor_points_per_race.append(round(row.driver_points / row['round'], 2))
    constructor_points_from_race.append(round(row.constructor_points_after_race - row.constructor_points, 1))

In [28]:
test['driver_points_per_race'] = driver_points_per_race
test['driver_points_from_race'] = driver_points_from_race

test['constructor_points_per_race'] = constructor_points_per_race
test['constructor_points_from_race'] = constructor_points_from_race

In [30]:
test.query('season == 2014 & round == 1')

,season,round,circuit_id,driver,constructor,podium,driver_points_after_race,driver_points,driver_wins,driver_standings_pos,...,constructor_standings_pos,qualifying_pos,qual_time,stage,q_delta,starting_grid,driver_points_per_race,driver_points_from_race,constructor_points_per_race,constructor_points_from_race
0,2014,1,albert_park,nico_rosberg,mercedes,1,25.0,0,0,0,...,0,3,104.595,q3,0.000,3,0.0,25.0,0.0,25.0
1,2014,1,albert_park,kevin_magnussen,mclaren,2,18.0,0,0,0,...,0,4,105.745,q3,-6.088,4,0.0,18.0,0.0,33.0
2,2014,1,albert_park,jenson_button,mclaren,3,15.0,0,0,0,...,0,11,104.437,q2,0.206,10,0.0,15.0,0.0,33.0
3,2014,1,albert_park,fernando_alonso,ferrari,4,12.0,0,0,0,...,0,5,105.819,q3,-6.014,5,0.0,12.0,0.0,18.0
4,2014,1,albert_park,valtteri_bottas,williams,5,10.0,0,0,0,...,0,10,108.147,q3,3.916,15,0.0,10.0,0.0,10.0
5,2014,1,albert_park,nico_hulkenberg,force_india,6,8.0,0,0,0,...,0,7,106.030,q3,-5.803,7,0.0,8.0,0.0,9.0
6,2014,1,albert_park,kimi_raikkonen,ferrari,7,6.0,0,0,0,...,0,12,104.494,q2,0.263,11,0.0,6.0,0.0,18.0
7,2014,1,albert_park,jean-eric_vergne,toro_rosso,8,4.0,0,0,0,...,0,6,105.864,q3,-5.969,6,0.0,4.0,0.0,6.0
8,2014,1,albert_park,daniil_kvyat,toro_rosso,9,2.0,0,0,0,...,0,8,107.368,q3,-4.465,8,0.0,2.0,0.0,6.0
9,2014,1,albert_park,sergio_perez,force_india,10,1.0,0,0,0,...,0,16,107.293,q2,3.062,16,0.0,1.0,0.0,9.0


In [80]:
round = 7
list(range(round-1, round-4, -1))

[6, 5, 4]

In [100]:
grouped = test.groupby(['season', 'round'])
last_3_races = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                rounds = list(range(round-1, round-4, -1))
                points = test.query(
                        'season ==@index[0] & round in @rounds & driver ==@row.driver'
                        ).agg({
                                'driver_points_from_race': sum
                                }).values[0]

                last_3_races.append(points)

In [97]:
grouped = test.groupby(['season', 'round'])
points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                query = test.query(
                        'season ==@index[0] & round ==@round-1 & driver ==@row.driver')

                points = query.driver_points_after_race
                if len(points) != 0:
                        points_before_race.append(points.values[0])
                else:
                        points_before_race.append(0.0)

In [101]:
test['driver_points_last_3_races'] = last_3_races
test['driver_points_before_race'] = points_before_race

In [107]:
testing = test[['season','round', 'driver', 'driver_points',  'driver_points_before_race', 'driver_points_from_race', 'driver_points_last_3_races']]

In [110]:
testing.query('season == 2021 & driver == "max_verstappen"')

,season,round,driver,driver_points,driver_points_before_race,driver_points_from_race,driver_points_last_3_races
2761,2021,1,max_verstappen,0,0.0,18.0,0.0
2780,2021,2,max_verstappen,18,18.0,25.0,18.0
2800,2021,3,max_verstappen,43,43.0,18.0,43.0
2820,2021,4,max_verstappen,61,61.0,19.0,61.0
2839,2021,5,max_verstappen,80,80.0,25.0,62.0
2874,2021,6,max_verstappen,105,105.0,0.0,62.0
2876,2021,7,max_verstappen,105,105.0,26.0,44.0
2894,2021,8,max_verstappen,131,131.0,25.0,51.0
2914,2021,9,max_verstappen,156,156.0,26.0,51.0
2953,2021,10,max_verstappen,182,182.0,3.0,77.0


In [111]:
merged.query('season == 2021 & round == 15')

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos,qualifying_pos,qual_time,stage,q_delta,starting_grid,driver_points_last_3_races
3030,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,lewis_hamilton,1985-01-07,British,...,1,362,4,1,4,104.050,q3,1.584,4,27.5
3031,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,carlos_sainz,1994-09-01,Spanish,...,4,201,0,4,2,102.510,q3,0.517,2,15.5
3032,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,daniel_ricciardo,1989-07-01,Australian,...,3,215,1,3,5,104.156,q3,1.690,5,33.0
3033,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,valtteri_bottas,1989-08-28,Finnish,...,1,362,4,1,7,104.710,q3,2.244,16,33.0
3034,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,fernando_alonso,1981-07-29,Spanish,...,5,95,1,5,6,104.204,q3,1.738,6,12.0
3035,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,lando_norris,1999-11-13,British,...,3,215,1,3,1,101.993,q3,0.000,1,19.0
3036,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,kimi_raikkonen,1979-10-17,Finnish,...,9,3,0,9,16,109.586,q1,7.593,13,0.0
3037,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,sergio_perez,1990-01-26,Mexican,...,2,344,8,2,9,105.337,q3,2.871,8,10.0
3038,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,george_russell,1998-02-15,British,...,8,22,0,8,3,102.983,q3,0.000,3,11.0
3039,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,lance_stroll,1998-10-29,Canadian,...,7,59,0,7,8,104.956,q3,2.490,7,6.0
